In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 100]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))

[ 1  2  3  4  5  6  7  8  9 10 11] [955054     14 479520     19   4005 467864      3   1083     52   2408
      2]
(array([ 1,  3,  5,  6,  8, 10]), array([955054, 479520,   4005, 467864,   1083,   2408]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '3' encode 1
defense '5' encode 2
defense '6' encode 3
defense '8' encode 4
defense '10' encode 5


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)

In [6]:
X_1.shape

(19100, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6] [     7 238746]
(7, 42) (7,)
Save 3 to baseline_Def3.npy
(238746, 42) (238746,)
Save 6 to baseline_Def6.npy
Execution Time: 0.334641 seconds
(716259,)
[ 1  3  5  6  8 10] [430740 252642  11578   9819      5  11475]
(430740, 42) (430740,)
Save 1 to BIM_Def1.npy
(252642, 42) (252642,)
Save 3 to BIM_Def3.npy
(11578, 42) (11578,)
Save 5 to BIM_Def5.npy
(9819, 42) (9819,)
Save 6 to BIM_Def6.npy
(5, 42) (5,)
Save 8 to BIM_Def8.npy
(11475, 42) (11475,)
Save 10 to BIM_Def10.npy
Execution Time: 0.552431 seconds
(716259,)
[ 1  3  5  6  8 10] [449126  43464  16521 196264   3529   7355]
(449126, 42) (449126,)
Save 1 to FGSM_Def1.npy
(43464, 42) (43464,)
Save 3 to FGSM_Def3.npy
(16521, 42) (16521,)
Save 5 to FGSM_Def5.npy
(196264, 42) (196264,)
Save 6 to FGSM_Def6.npy
(3529, 42) (3529,)
Save 8 to FGSM_Def8.npy
(7355, 42) (7355,)
Save 10 to FGSM_Def10.npy
Execution Time: 0.463591 seconds
(716259,)
[ 1  3  5  6  8 10] [430740 252642  11578   9819      5  11475]
(430740, 42) (430740,)


In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6] [     8 238745]
(8, 42) (8,)
Save 3 to baseline_Def3.npy
(238745, 42) (238745,)
Save 6 to baseline_Def6.npy
Execution Time: 0.542132 seconds
(716259,)
[ 1  3  5  6 10] [220529 490981   3246   1448     55]
(220529, 42) (220529,)
Save 1 to BIM_Def1.npy
(490981, 42) (490981,)
Save 3 to BIM_Def3.npy
(3246, 42) (3246,)
Save 5 to BIM_Def5.npy
(1448, 42) (1448,)
Save 6 to BIM_Def6.npy
(55, 42) (55,)
Save 10 to BIM_Def10.npy
Execution Time: 2.184591 seconds
(716259,)
[ 1  3  5  6 10] [443130 237871  20606  14584     68]
(443130, 42) (443130,)
Save 1 to FGSM_Def1.npy
(237871, 42) (237871,)
Save 3 to FGSM_Def3.npy
(20606, 42) (20606,)
Save 5 to FGSM_Def5.npy
(14584, 42) (14584,)
Save 6 to FGSM_Def6.npy
(68, 42) (68,)
Save 10 to FGSM_Def10.npy
Execution Time: 2.081166 seconds
(716259,)
[ 1  3  5  6 10] [220529 490981   3246   1448     55]
(220529, 42) (220529,)
Save 1 to PGD_Def1.npy
(490981, 42) (490981,)
Save 3 to PGD_Def3.npy
(3246, 42) (3246,)
Save 5 to PGD_Def5.npy
(1448, 42)

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ExcludeCaFA.csv")

In [13]:
all_attack_list

0       1   2       3   4      5       6   7     8   9      10  11  \
0   XGB1       0   0       7   0      0  238746   0     0   0      0   0   
1   XGB1  430740   0  252642   0  11578    9819   0     5   0  11475   0   
2   XGB1  449126   0   43464   0  16521  196264   0  3529   0   7355   0   
3   XGB1  430740   0  252642   0  11578    9819   0     5   0  11475   0   
4   XGB1     117   0    6443   0  11804  687474   0  3411   0   7010   0   
5   XGB1  155615   0  525898   0   6382   18125   0  1605   0   8634   0   
6   XGB1       0   0      27   0      0  716232   0     0   0      0   0   
7   XGB1   54439   0  321712   0   2909  336869   0     0   0    330   0   
8   XGB1    3442   0  696998   0   7463    1347   0     0   0   7009   0   
9   XGB1  426048   0  104665   0  17509  167359   0   678   0      0   0   
10   RF1       0   0       8   0      0  238745   0     0   0      0   0   
11   RF1  220529   0  490981   0   3246    1448   0     0   0     55   0   
12   RF1  443130   0  237871   0  20606   14584   0     0   0     68   0   
13   RF1  220529   0  490981   0   3246    1448   0     0   0     55   0   
14   RF1     135   0    7979   0   9748  688091   0  3557   0   6749   0   
15   RF1  130282   0  522541   0    270   63166   0     0   0      0   0   
16   RF1       0   0      23   0      0  716236   0     0   0      0   0   
17   RF1     569   0  467952   0      0  247738   0     0   0      0   0   
18   RF1    1271   0  711202   0    139    3647   0     0   0      0   0   
19   RF1  415783   0  275820   0  17145    7511   0     0   0      0   0   
20   DT1       0   0       0   0      0       0   0     0   0      0   0   
21   DT1       0   0       0   0      0       0   0     0   0      0   0   
22   DT1       0   0       0   0      0       0   0     0   0      0   0   
23   DT1       0   0       0   0      0       0   0     0   0      0   0   
24   DT1       0   0       0   0      0       0   0     0   0      0   0   
25   DT1       0   0       0   0      0       0   0     0   0      0   0   
26   DT1       0   0       0   0      0       0   0     0   0      0   0   
27   DT1       0   0       0   0      0       0   0     0   0      0   0   
28   DT1       0   0       0   0      0       0   0     0   0      0   0   
29   DT1       0   0       0   0      0       0   0     0   0      0   0   

        12  
0   238753  
1   716259  
2   716259  
3   716259  
4   716259  
5   716259  
6   716259  
7   716259  
8   716259  
9   716259  
10  238753  
11  716259  
12  716259  
13  716259  
14  716259  
15  716259  
16  716259  
17  716259  
18  716259  
19  716259  
20       0  
21       0  
22       0  
23       0  
24       0  
25       0  
26       0  
27       0  
28       0  
29       0